In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dataset = pd.read_csv("../data/01_raw/CarPrice.csv")
dataset.head()


,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [62]:
dataset.shape

(205, 26)

In [63]:
data = dataset.sample(frac=0.9)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print("Data for modeling: " + str(data.shape))
print("Unseen data for predictions: " + str(data_unseen.shape))

Data for modeling: (184, 26)
Unseen data for predictions: (21, 26)


In [64]:
from pycaret.regression import *
regress = setup(data = data, target = "price", session_id = 69, use_gpu = True, train_size=0.8)

,Description,Value
0,Session id,69
1,Target,price
2,Target type,Regression
3,Data shape,"(184, 32)"
4,Train data shape,"(147, 32)"
5,Test data shape,"(37, 32)"
6,Ordinal features,4
7,Numeric features,15
8,Categorical features,10
9,Preprocess,True


In [65]:
#evaluate models
best_model = compare_models(fold = 15, sort="MAE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
llar,Lasso Least Angle Regression,2063.3166,8056753.4732,2653.0119,0.7529,0.2070,0.1698,0.0647
ada,AdaBoost Regressor,2064.1974,7765053.2596,2626.0152,0.8220,0.1939,0.1687,0.1067
ridge,Ridge Regression,2085.2924,7854422.2472,2642.2469,0.7736,0.2199,0.1705,0.0600
lasso,Lasso Regression,2130.6487,8465907.7011,2719.5522,0.7378,0.2121,0.1769,0.0653
lr,Linear Regression,2137.0519,8530010.3654,2726.3944,0.7351,0.2121,0.1777,0.0600
et,Extra Trees Regressor,2142.2659,8395945.8599,2675.5508,0.8451,0.1883,0.1727,0.1427
gbr,Gradient Boosting Regressor,2308.9253,8813817.5747,2823.2669,0.7973,0.2115,0.1925,0.1040
en,Elastic Net,2354.1582,9678907.9716,2981.5222,0.7188,0.2408,0.1870,0.0620
br,Bayesian Ridge,2431.1058,10482961.2293,3098.4033,0.7075,0.2312,0.1865,0.0607
omp,Orthogonal Matching Pursuit,2627.0767,13717131.2273,3510.4201,0.5970,0.2515,0.1993,0.0607


In [66]:
gbr = create_model('gbr', fold = 15)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2036.3621,5296308.2398,2301.3709,0.9295,0.1675,0.1604
1,2704.4219,9443885.1765,3073.0905,0.7658,0.2847,0.2854
2,3378.4723,24563837.4380,4956.1918,0.8057,0.1927,0.1655
3,2344.0106,8688543.9949,2947.6336,0.8867,0.2311,0.2060
4,3229.1029,13078330.9612,3616.3975,0.8033,0.2642,0.2516
5,3164.2279,18379949.6489,4287.1843,0.8426,0.1845,0.1604
6,1423.9139,3083883.6538,1756.0990,0.7814,0.2086,0.1796
7,1759.4245,4200002.4615,2049.3908,0.9631,0.2234,0.1866
8,2764.7916,10989031.0651,3314.9707,0.9191,0.2426,0.2167


In [67]:
tuned_gbr = tune_model(gbr, optimize="MAE", n_iter=100, fold = 15)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1351.2280,2441095.0363,1562.4004,0.9675,0.1403,0.1249
1,2312.1693,7582960.7964,2753.7176,0.8119,0.2455,0.2349
2,2402.4208,10800228.3345,3286.3701,0.9146,0.1553,0.1323
3,2518.6420,10857433.1758,3295.0619,0.8584,0.2316,0.1988
4,2755.1346,10646429.9458,3262.8868,0.8399,0.2113,0.1874
5,2881.6694,18987558.9878,4357.4716,0.8374,0.1726,0.1370
6,1168.4566,1985054.3728,1408.9196,0.8593,0.1786,0.1504
7,1942.2414,5616138.3988,2369.8393,0.9507,0.2055,0.1657
8,1443.3683,3018183.3227,1737.2920,0.9778,0.1509,0.1254


Fitting 15 folds for each of 100 candidates, totalling 1500 fits


In [68]:
evaluate_model(tuned_gbr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [69]:
predict_model(tuned_gbr);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,1513.5511,4222021.5853,2054.7558,0.8610,0.1597,0.1411


In [70]:
final_gbr = finalize_model(tuned_gbr)

In [71]:
print(final_gbr)

Pipeline(memory=Memory(location=C:\Users\tyveg\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['car_ID', 'symboling', 'wheelbase',
                                             'carlength', 'carwidth',
                                             'carheight', 'curbweight',
                                             'enginesize', 'boreratio',
                                             'stroke', 'compressionratio',
                                             'horsepower', 'peakrpm', 'citympg',
                                             'highwaympg'],
                                    transformer=SimpleImputer())),
                ('categor...
                                                                   handle_missing='return_nan',
                                                                   random_state=69))),
                ('low_variance',
                 TransformerWrapper(exclude=[],
                

In [72]:
unseen_predictions = predict_model(final_gbr, data=data_unseen)
unseen_predictions.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,956.7841,1898780.3340,1377.9624,0.9621,0.0840,0.0663


,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody_hatchback,carbody_sedan,carbody_wagon,carbody_hardtop,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,prediction_label
0,4.0,2.0,13376.725543,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,17905.398810,3.19,3.40,10.0,102.0,5500.0,24.0,30.0,13950.000000,11840.127946
1,10.0,0.0,13376.725543,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,17858.861096,3.13,3.40,7.0,160.0,5500.0,16.0,22.0,17859.167969,20166.621781
2,23.0,1.0,13376.725543,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,7518.627119,2.97,3.23,9.4,68.0,5500.0,31.0,38.0,6377.000000,6174.222667
3,39.0,0.0,13376.725543,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,7213.444444,3.15,3.58,9.0,86.0,5800.0,27.0,33.0,9095.000000,8978.813536
4,50.0,0.0,13376.725543,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,17642.898810,3.54,2.76,11.5,262.0,5000.0,13.0,17.0,36000.000000,34766.587412
